# 유튜브 인기동영상 데이터

## 데이터 로드

In [1]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv",index_col=0)
df.head()

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01


## 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력(날짜 기준, 중복 포함)

In [2]:
print(df.loc[df.channelId.isin(list(df.groupby('channelId').size().sort_values(ascending=False).keys()[:10]))].channelTitle.unique())

['장삐쭈' '총몇명' '파뿌리' '짤툰' '런닝맨 - 스브스 공식 채널' '엠뚜루마뚜루 : MBC 공식 종합 채널' 'SPOTV'
 '채널 십오야' '이과장' 'BANGTANTV']


## 논란으로 인기동영상이 된 케이스 확인, dislikes 수가 like 수보다 높은 동영상을 제작한 채널을 모두 출력

In [3]:
print(df.loc[df.dislikes > df.likes].channelTitle.unique())

['핫도그TV' 'ASMR 애정TV' '하얀트리HayanTree' '양팡 YangPang' '철구형 (CHULTUBE)'
 '왜냐맨하우스' '(MUTUBE)와꾸대장봉준' '오메킴TV' '육지담' 'MapleStory_KR'
 'ROAD FIGHTING CHAMPIONSHIP' '사나이 김기훈' '나혼자산다 STUDIO' 'Gen.G esports']


## 채널명을 한번이라도 바꾼 채널의 갯수

In [4]:
change = df[['channelId','channelTitle']].drop_duplicates().channelId.value_counts()
print(len(change[change>1]))

71


## 일요일에 인기있었던 영상들 중 가장 많은 영상 종류

In [5]:
# df['weekends'] = df['trending_date2'].dt.weekends
df['trending_date2'] = pd.to_datetime(df['trending_date2'])
df['weekday'] = df['trending_date2'].dt.day_name()
print(df.loc[df.weekday=='Sunday'].categoryId.value_counts().index[0])

24


## 각 요일별 인기 영상들의 categoryID는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라

In [6]:
print(pd.pivot_table(df,index='categoryId',columns='weekday',aggfunc='size'))

weekday     Friday  Monday  Saturday  Sunday  Thursday  Tuesday  Wednesday
categoryId                                                                
1              243     263       255     274       246      257        234
2              120     105       119      99       128      119        129
10             833     837       776     830       890      894        917
15             187     215       198     217       207      208        207
17             633     668       592     636       682      708        706
19              90      92        87      91        92       89         85
20             283     298       296     289       282      285        291
22            1288    1373      1289    1337      1341     1375       1333
23             568     594       570     556       560      569        566
24            2976    3148      3066    3096      2954     3084       3090
25             444     453       422     437       470      452        468
26             369     37

## viewcount 대비 댓글 수가 가장 높은 영상을 확인

In [11]:
quest = df.loc[df.view_count != 0]
quest['feedback'] = quest['comment_count']/quest['view_count'].dropna()

quest.sort_values('feedback',ascending =False).iloc[0]['title']

/tmp/ipykernel_762/1598414501.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quest['feedback'] = quest['comment_count']/quest['view_count'].dropna()


'60분 동안 댓글이 달리지 않으면, 영상이 삭제됩니다. (챌린지)'

## viewcount 대비 댓글 수가 가장 높은 영상을 확인

In [22]:
quest = df.loc[df.view_count !=0]
quest['feedback'] = (quest.comment_count/quest.view_count)
quest = quest.loc[quest.feedback != 0]
quest.sort_values('feedback').iloc[0]['title']

/tmp/ipykernel_762/2899900271.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quest['feedback'] = (quest.comment_count/quest.view_count)


'Join the BTS #PermissiontoDance Challenge only on YouTube #Shorts'

## 좋아요 수 대비 싫아요 비율이 가장 낮은 영상 확인

In [23]:
quest = df.loc[(df.likes != 0) & (df.dislikes != 0)]
quest['ratio']=df.dislikes/df.likes
quest.sort_values('ratio').iloc[0]['title']

/tmp/ipykernel_762/2567045718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quest['ratio']=df.dislikes/df.likes


'[줌터뷰] *최초공개* 사부작즈🐰🐶의 비공식 이름은 아이라인즈? 꿀조합 티키타카 가득한 NCT 127 도영&정우의 줌터뷰'

## 가장 많은 트렌드 영상을 제작한 채널의 이름

In [24]:
print(df.loc[df.channelId == df.channelId.value_counts().index[0]].channelTitle.unique()[0])

짤툰


## 20회 이상 인기동영상 리스트에 포함된 동영상의 숫자

In [25]:
print((df[['title','channelId']].value_counts() >= 20).sum())


40


# 유튜브 공범컨텐츠 동영상 데이터

## 데이터 로드

In [26]:
import pandas as pd
channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')

## 각 데이터의 'ct' 컬럼을 시간으로 인식할 수 있게 datatype을 변경하고 video 데이터의 videoname의 각 value 마다 몇 개의 데이터씩 가지고 있는지 확인

In [32]:
channel['ct'] = pd.to_datetime(channel['ct'])
video['ct'] = pd.to_datetime(video['ct'])
print(video['videoname'].value_counts())

videoname
공범 EP1    3492
공범 EP2    3204
공범 EP3    2568
공범 EP4    2280
공범 EP5    1562
공범 EP6    1274
공범 EP7     555
공범 EP8     266
Name: count, dtype: int64


## 수집된 각 vide의 가장 최신화된 날짜의 viewcount 값 출력

In [43]:
print(video[['viewcnt','videoname','ct']].sort_values(['videoname','ct']).drop_duplicates('videoname',keep='last').reset_index(drop=True))

   viewcnt videoname                  ct
0  3180532    공범 EP1 2021-11-01 15:30:03
1  2199328    공범 EP2 2021-11-01 15:30:03
2  1671294    공범 EP3 2021-11-01 15:30:03
3  1818493    공범 EP4 2021-11-01 15:30:03
4  1503435    공범 EP5 2021-11-01 15:30:04
5  1750222    공범 EP6 2021-11-01 15:30:04
6  1630200    공범 EP7 2021-11-01 15:30:05
7  1289088    공범 EP8 2021-11-01 15:30:05


## channel 데이터 중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수 출력

In [51]:
quest = channel.loc[channel.ct >= '2021-10-03'].sort_values(['channelid','ct']).drop_duplicates('channelid',keep='first')
print(quest[['channelname','subcnt']].reset_index(drop=True))

        channelname   subcnt
0            논리왕 전기   922000
1             야전삽짱재   257000
2           형사!탐정되다    14900
3       릴펄 Lilpearl    10100
4               조나단   215000
5        김농밀의 농밀한 삶     7520
6     Balming Tiger    54300
7               꽈뚜룹  1330000
8   와글와글 WagleWagle    55000
9        츄정ChuJeong   322000
10    곽토리 kwak tori   471000


## 각 채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수의 증가량

In [74]:
before = channel.loc[channel.ct >= '2021-10-03 03:00:00'].sort_values(['channelid','ct']).drop_duplicates('channelid',keep='first')
before = before.rename(columns={'subcnt':'before'})
after = channel.loc[channel.ct >= '2021-11-01 15:00:00'].sort_values(['channelid','ct']).drop_duplicates('channelid',keep='last')
after = after.rename(columns={'subcnt':'after'})
after
total = pd.merge(before[['channelid','channelname','before']],after[['channelid','channelname','after']])
total['del'] = total['after'] - total['before']
print(total[['channelname','del']])

        channelname    del
0            논리왕 전기 -11000
1             야전삽짱재  11000
2           형사!탐정되다  10300
3       릴펄 Lilpearl  11000
4               조나단  12000
5        김농밀의 농밀한 삶   1540
6     Balming Tiger   2500
7               꽈뚜룹  70000
8   와글와글 WagleWagle      0
9        츄정ChuJeong   1000
10    곽토리 kwak tori  -2000


## 각 비디오는 10분 간격으로 구독자 수, 좋아요, 싫어요 수, 댓글 수 가 수집됨. 공범 EP1의 비디오 정보 데이터 중 수집간격이 5분 이하, 20분 이상인 데이터 구간(해당 시점 전, 후)의 시각을 모두 출력

In [117]:
part1 = video.loc[video['videoname'] == ' 공범 EP1'].sort_values('ct')
part1.loc[(part1.ct.diff(1) <= '0 days 00:5:00')|(part1.ct.diff(1) >= '0 days 00:20:00')]
print(part1.loc[[720,721,722,723,1635,1636,1637],:])

          videopk  viewcnt  likecnt  dislikecnt  favoritecnt  cmcnt  \
720   c5JQp6xafqc  2153676    34104         830            0   7120   
721   c5JQp6xafqc  2228250    34559         849            0   7191   
722   c5JQp6xafqc  2228250    34559         849            0   7191   
723   c5JQp6xafqc  2228669    34566         849            0   7194   
1635  c5JQp6xafqc  2706691    37893         987            0   7410   
1636  c5JQp6xafqc  2707933    37901         988            0   7410   
1637  c5JQp6xafqc  2708448    37903         988            0   7410   

                      ct videoname  
720  2021-10-12 19:20:03    공범 EP1  
721  2021-10-13 09:41:37    공범 EP1  
722  2021-10-13 09:41:37    공범 EP1  
723  2021-10-13 09:50:03    공범 EP1  
1635 2021-10-19 17:50:02    공범 EP1  
1636 2021-10-19 18:20:03    공범 EP1  
1637 2021-10-19 18:30:03    공범 EP1  


## 각 에피소드의 시작날짜(년-월-일)을 에피소드 이름과 묶어 데이터 프레임으로 만들고 출력

In [133]:
video['date'] = video['ct'].dt.date
result = video.sort_values(['videoname','date']).drop_duplicates('videoname',keep='first')
print(result[['videoname','date']])


      videoname        date
10       공범 EP1  2021-10-07
3496     공범 EP2  2021-10-09
6696     공범 EP3  2021-10-14
9264     공범 EP4  2021-10-16
11544    공범 EP5  2021-10-21
13661    공범 EP6  2021-10-23
13106    공범 EP7  2021-10-28
14935    공범 EP8  2021-10-30


## video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 커럼으로 만들고 videoname,ratio로 구성된 데이터 프레임을 ratio 오름차순으로 정렬

In [142]:
result = video.sort_values(['videoname','ct']).drop_duplicates('videoname',keep='last')
result['ratio'] = result.dislikecnt/result.likecnt
print(result[['videoname','ratio']].sort_values('ratio'))

      videoname     ratio
13660    공범 EP7  0.013636
14934    공범 EP6  0.015881
11543    공범 EP4  0.017971
13105    공범 EP5  0.021128
9263     공범 EP3  0.022360
3491     공범 EP1  0.027118
6695     공범 EP2  0.029844
15200    공범 EP8  0.141357


## 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오

In [153]:
before = video.loc[(video.ct >= '2021-11-01 00:00:00'),['videoname','viewcnt','ct']].sort_values(['videoname','ct']).drop_duplicates('videoname',keep='first')
before = before.rename(columns={'viewcnt':'before'})
after = video.loc[(video.ct <= '2021-11-01 15:00:00'),['videoname','viewcnt','ct']].sort_values(['videoname','ct']).drop_duplicates('videoname',keep='last')
after = after.rename(columns={'viewcnt':'after'})
total = pd.merge(before[['videoname','before']],after[['videoname','after']])
total['viewcnt'] = total['after']-total['before']
print(total[['videoname','viewcnt']])

  videoname  viewcnt
0    공범 EP1    13298
1    공범 EP2    10300
2    공범 EP3     9927
3    공범 EP4     9824
4    공범 EP5    10824
5    공범 EP6    14141
6    공범 EP7    26949
7    공범 EP8    89147


## video 데이터 중에서 중복되는 데이터의 시간대와 videoname

In [174]:
answer = video[video.index.isin(set(video.index) - set(video.drop_duplicates().index))]
print(answer[['videoname','ct']])

     videoname                  ct
722     공범 EP1 2021-10-13 09:41:37
3927    공범 EP2 2021-10-13 09:41:37


# 월드컵 출전선수 골기록 데이터

## 데이터 로드

In [410]:
import pandas as pd

df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv')
df.head()

,Player,Goals,Years,Country
0,Miroslav Klose,16,2002-2006-2010-2014,Germany
1,Ronaldo,15,1998-2002-2006,Brazil
2,Gerd Muller,14,1970-1974,Germany
3,Just Fontaine,13,1958,France
4,Pele,12,1958-1962-1966-1970,Brazil


## 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 그 득점수 출력

In [411]:
print(df[['Country','Goals']].groupby('Country').sum().sort_values('Goals',ascending=False).head(5))


           Goals
Country         
Brazil       228
Germany      226
Argentina    135
Italy        123
France       115


## 골득점을 한 선수가 많은 상위 5개 국가와 그 선수 숫자를 데이터 프레임 형식으로 출력

In [412]:
print(df[['Country','Player']].groupby('Country').size().sort_values(ascending=False).head(5))

Country
Brazil       81
Germany      78
Italy        60
Argentina    59
France       58
dtype: int64


## 년도 표기가 4자리 숫자로 안된 케이스가 몇 건인지 출력

In [413]:
df['Years_list'] = df['Years'].str.split('-')
def check(row):
    answer = False
    for i in row:
        if len(i) != 4:
            answer = True
    return answer
df['Check'] = df['Years_list'].apply(check)
print(len(df.loc[df['Check'] == True]))

45


## 년도 표기가 4자리 숫자로 되어있는 케이스만 추출

In [414]:
df2 = df.loc[df['Check'] == False]
print(len(df2))

1250


## 월드컵 출전횟수를 나타내는 'LenCup' 컬럼을 추가하고 4회 출전한 선수의 숫자 출력

In [415]:
df2['LenCup'] = df.Years_list.map(lambda x : len(x))
print(len(df2.loc[df2.LenCup == 4]))

16


/tmp/ipykernel_762/715742222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['LenCup'] = df.Years_list.map(lambda x : len(x))


## Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자

In [416]:
print(len(df2[(df2.Country == 'Yugoslavia') & (df2.LenCup == 2)]))

7


## 2002년도에 출전한 전체 선수 

In [417]:
print(len(df2.loc[df2.Years.str.contains('2002')]))

156


## 이름에 'carlos' 단어가 들어가는 선수의 숫자

In [418]:
print(len(df2.loc[df.Player.str.lower().str.contains('carlos')]))

13


## 월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수

In [419]:
print(df2.loc[df2.LenCup == 1].sort_values('Goals',ascending=False)['Player'].values[0])

Just Fontaine


## 월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가

In [420]:
print(df2.loc[df2.LenCup == 1].groupby('Country').size().sort_values(ascending=False).index[0])

Brazil


# 서울시 따릉이 이용정보 데이터

## 데이터 로드

In [421]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')
df.head()

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


## 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜 출력

In [422]:
print(df.groupby('대여일자').size())
print(df.groupby('대여일자').size().sort_values(ascending= False).index[0])

대여일자
2021-06-01    48081
2021-06-02    56100
2021-06-03    18985
2021-06-04    56788
2021-06-05    52730
2021-06-06    47787
2021-06-07    52623
2021-06-08    55623
2021-06-09    54507
2021-06-10    36992
2021-06-11    52089
2021-06-12    48569
2021-06-13    43131
2021-06-14    54945
dtype: int64
2021-06-04


## 각 일자의 요일을 표기하고 day_name 칼럼을 추가하고 이를 이용하여 각 요일별 이용 회숫의 총합을 데이터 프레임으로 출력

In [423]:
df['대여일자'] = pd.to_datetime(df['대여일자'])
df['day_name'] = df['대여일자'].dt.day_name()
print(df.groupby('day_name').size())

day_name
Friday       108877
Monday       107568
Saturday     101299
Sunday        90918
Thursday      55977
Tuesday      103704
Wednesday    110607
dtype: int64


## 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력

In [424]:
df.groupby(['day_name','대여소번호']).size().sort_values(ascending=False).reset_index().drop_duplicates('day_name',keep='first')

,day_name,대여소번호,0
0,Saturday,502,378
1,Sunday,502,372
8,Wednesday,502,282
9,Friday,502,277
10,Tuesday,502,267
19,Monday,502,242
298,Thursday,2715,137


## 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대 확인/일일권의 경우 일일권과 일일권(비회원)을 모두 포함

In [425]:
print((df.loc[df['대여구분코드'].str.contains('일일권')].groupby('연령대코드').size() /df.groupby('연령대코드').size()))
print((df.loc[df['대여구분코드'].str.contains('일일권')].groupby('연령대코드').size() /df.groupby('연령대코드').size()).sort_values().index[-1])

연령대코드
20대     0.239913
30대     0.166702
40대     0.110585
50대     0.065414
60대     0.042462
70대~    0.048962
~10대    0.312525
dtype: float64
~10대


## 연령대별 평균 이동거리

In [426]:
print(df.groupby('연령대코드').이동거리.mean())

연령대코드
20대     3211.890552
30대     3341.443859
40대     3514.857416
50대     3593.668100
60대     3538.145737
70대~    3085.039641
~10대    2634.426279
Name: 이동거리, dtype: float64


## 연령대가 20대인 데이터를 추출하고, 이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출, 최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하기

In [427]:
answer = df.loc[df.연령대코드 == '20대'].loc[df.이동거리 >= df.이동거리.mean()].sort_values(['대여일자','대여소번호'],ascending=False).reset_index(drop=True)[:200]
answer['탄소량'] = answer['탄소량'].astype(float)
print(round(answer['탄소량'].mean(),3))

1.643


## 6월 7일 ~10의 이용건수의 중앙값

In [428]:
df.loc[(df.대여일자 == '2021-06-07')&(df.연령대코드 == '~10대')].이용건수.median()

1.0

## 평일 출근 시간대의 대여소별 이용 횟수 추출/ 대여시간별 이용횟수의 상위 3개 대여소와 이용횟수

In [429]:
result = df.loc[(~df.day_name.isin(['Saturday','Sunday']))&(df.대여시간.isin([6,7,8]))].groupby(['대여시간','대여소번호']).size().to_frame('이용 횟수')
print(result.sort_values(['대여시간','이용 횟수'],ascending=False).groupby('대여시간').head(3))

            이용 횟수
대여시간 대여소번호       
8    2701     119
     646      115
     1152      92
7    259      104
     230       77
     726       77
6    2744      45
     1125      40
     1028      36


## 이동거리 평균 이상의 이동거리 값을 가지는 데이터의 이동거리 표본표준편차

In [430]:
import numpy as np
print(df.loc[df.이동거리 >= df.이동거리.mean()].이동거리.std())

5092.139707505356


## 남성와 여성의 이동거리값의 평균값

In [432]:
def sex(row):
    if row == 'F' or row == 'f':
        answer = '여성'
    else:
        answer = '남성'
    return answer
df['sex'] = df['성별'].apply(sex)
df
df.groupby('sex').이동거리.mean()

sex
남성    3209.110871
여성    3468.575025
Name: 이동거리, dtype: float64

# 전세계 행복도 지표 데이터

## 데이터 로드

In [433]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv',encoding='utf-8')
df.head()

,행복랭킹,나라명,점수,상대GDP,사회적지원,행복기대치,선택의 자유도,관대함,부패에 대한인식,년도
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,2019
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,2019
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,2019


## 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현하고, 각 년도의 행복랭킹 50위 이내의 나라들의 각각의 행복 점수 평균을 데이터 프레임으로 표시

In [446]:
answer = df.loc[(df.년도 == 2018)|(df.년도 == 2019)].sort_values(['년도','점수'],ascending=False).groupby('년도').head(50)
print(answer.groupby('년도').점수.mean().to_frame())

           점수
년도           
2018  6.64678
2019  6.67002


## 2018년도 데이터들만 추출하여 행복점수와 부패에 대한 인식에 대한 상관계수를 구하기

In [457]:
print(df.loc[df.년도 == 2018][['점수','부패에 대한인식']].corr().iloc[0,1])

0.40529152271510027


## 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수

In [484]:
df2018 = df.loc[df.년도 == 2018]
df2018 = df2018.rename(columns={'년도':'2018','행복랭킹':'2018 점수'})
df2019 = df.loc[df.년도 == 2019]
df2019 = df2019.rename(columns={'년도':'2019','행복랭킹':'2019 점수'})
df2019 =df2019[['나라명','2019 점수']]
df2018 = df2018[['나라명','2018 점수']]
total = pd.merge(df2018,df2019)
total['diff'] = total['2019 점수'] - total['2018 점수']
# total.loc[total.diff == 0]
print(len(total.loc[total['diff']==0]))

15


## 2019년도 데이터들만 추출하여 각 변수간 상관계수를 구하고 내림차순으로 정렬한 후 상위 5개를 데이터 프레임으로 출력, 컬렴명 v1, v2, corr 표시

In [501]:
df2019 = df.loc[df.년도 == 2019]
result = df2019[['상대GDP','사회적지원','행복기대치','선택의 자유도','관대함','부패에 대한인식']].corr().unstack().reset_index().drop_duplicates()
result = result.rename(columns={'level_0':'v1','level_1':'v2',0:'corr'})
print(result.loc[result['corr'] != 1].sort_values('corr',ascending=False).reset_index(drop=True).head(5))

      v1     v2      corr
0  상대GDP  행복기대치  0.835462
1  행복기대치  상대GDP  0.835462
2  상대GDP  사회적지원  0.754906
3  사회적지원  상대GDP  0.754906
4  사회적지원  행복기대치  0.719009


## 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수

In [511]:
result = df.sort_values(['년도','점수']).groupby('년도').head(5)
print(result.groupby('년도').점수.mean())

년도
2018    3.1800
2019    3.1408
Name: 점수, dtype: float64


## 각 년도의 부패에 대한 인식을 내림차순 정렬했을 때 상위 20개 국가의 부패에 대한 인식의 평균

In [515]:
result = df.sort_values(['년도','부패에 대한인식'],ascending=False).groupby('년도').head(20)
print(result.groupby('년도')['부패에 대한인식'].mean())

년도
2018    0.3267
2019    0.3201
Name: 부패에 대한인식, dtype: float64


## 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자

In [519]:
df2018 = df.loc[(df.행복랭킹 <= 50)&(df.년도 == 2018)]
df2019 = df.loc[(df.행복랭킹 >= 50)&(df.년도 == 2019)]
total = pd.merge(df2018['나라명'],df2019['나라명'])
print(len(total))

4


## 2018년, 2019년 모두 기록이 있는 나라들 중 년도별 행복점수가 가장 증가한 나라와 그 증가 수치

In [529]:
df2018 = df.loc[df.년도 == 2018]
df2019 = df.loc[df.년도 == 2019]
df2018 = df2018[['나라명','점수']]
df2018 = df2018.rename(columns={'점수':'2018점수'})
df2019 = df2019[['나라명','점수']]
df2019 = df2019.rename(columns={'점수':'2019점수'})
total = pd.merge(df2018,df2019)
total['diff'] = total['2019점수'] - total['2018점수']
total.sort_values('diff',ascending=False).head(1)

,나라명,2018점수,2019점수,diff
151,Burundi,2.905,3.775,0.87


# 지역구 에너지 소비량 데이터

## 데이터 로드

In [530]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan%20City%20power%20consumption.csv')
df.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


## dataframe 컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 출력

In [536]:
df['month'] = pd.to_datetime(df.DateTime).dt.month
print(df.groupby('month').size())

month
1     4464
2     4032
3     4464
4     4320
5     4464
6     4320
7     4464
8     4464
9     4320
10    4464
11    4320
12    4320
dtype: int64


## 3월달의 각 시간대별 온도 평균들 중 가장 낮은 시간대의 온도 출력

In [549]:
df['hour'] = pd.to_datetime(df.DateTime).dt.hour
print(df.loc[df.month == 3].groupby('hour').Temperature.mean().sort_values().head(1))

hour
7    11.506613
Name: Temperature, dtype: float64


## 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도 출력

In [551]:
df['hour'] = pd.to_datetime(df.DateTime).dt.hour
print(df.loc[df.month == 3].groupby('hour').Temperature.mean().sort_values(ascending=False).head(1))

hour
15    18.393602
Name: Temperature, dtype: float64


## Zone 1 Power Consumption 컬럼의 value 값의 크기가 Zone 2 Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 Humidity의 평균

In [563]:
df['diff'] = df['Zone 1 Power Consumption'] - df['Zone 2  Power Consumption']
print(df.loc[df['diff'] > 0].Humidity.mean())

68.23624448055052


## 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기

In [570]:
df[['Zone 1 Power Consumption','Zone 2  Power Consumption','Zone 3  Power Consumption']].corr()

,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
Zone 1 Power Consumption,1.000000,0.834519,0.750733
Zone 2 Power Consumption,0.834519,1.000000,0.570932
Zone 3 Power Consumption,0.750733,0.570932,1.000000


## Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B, 20 이상 30 미만의 경우 C, 그 외의 경우 D라고 할 때 각 단계의 데이터 숫자

In [575]:
def temperature(row):
    if row < 10:
        answer = "A"
    elif row >= 10 and row < 20:
        answer = 'B'
    elif row >= 20 and row < 30:
        answer = 'C'
    else:
        answer = 'D'
    return answer
df['grade'] = df['Temperature'].apply(temperature)
print(df.groupby('grade').size())

grade
A     2874
B    26993
C    21105
D     1444
dtype: int64


## 6월 데이터 중 12시의 Temperature의 표준편차

In [578]:
df.loc[(df['month'] == 6)&(df['hour'] == 12)].Temperature.std()

2.0499417827951025

## 6월 데이터 중 12시의 Temperature의 분산

In [580]:
print(df.loc[(df['month'] == 6)&(df['hour'] == 12)].Temperature.var())

4.202261312849163


## Temperature의 평균 이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을 때 4번째 행의 Humidity

In [587]:
print(df.loc[df['Temperature']>= df['Temperature'].mean()].sort_values('Temperature').reset_index(drop=True).loc[3,'Humidity'])

87.9


## Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬했을 때 4번째 행의 Humidity

In [589]:
print(df.loc[df['Temperature']>= df['Temperature'].median()].sort_values('Temperature').reset_index(drop=True).loc[3,'Humidity'])

80.3


# 포켓몬 정보 데이터